In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, Activation, BatchNormalization, GRU, LSTM, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
import matplotlib.pyplot as plt

# Set seed for reproducibility
np.random.seed(2)

In [ ]:
# Load dataset for electrode 1 (E1)
dataset_e1= pd.read_csv("/content/drive/MyDrive/Code_IA_Test/E1.csv",delimiter=",")
dataset_e1=np.array(dataset_e1,float)
X_e1 = dataset_e1[:,1:6]
# Load dataset for electrode 2 (E2)
dataset_e2= pd.read_csv("/content/drive/MyDrive/Code_IA_Test/E2.csv",delimiter=",")
dataset_e2=np.array(dataset_e2,float)
X_e2 = dataset_e2[:,1:6]
# Extract output (Y)
Y= dataset_e1[:,8]
print(Y.shape)

In [ ]:
# Normalize data
# Step1
lin,col =np.shape(X_e1)
for i in range(0, col-2):
    X_e1[:, i] = (X_e1[:, i]/dataset_e1[:, 0])
    X_e2[:, i] = (X_e2[:, i]/dataset_e2[:, 0])

# Step 2: Use StandardScaler for further normalization
scaler1 = StandardScaler().fit(X_e1)
scaler2 = StandardScaler().fit(X_e2)
X_e1 = scaler1.transform(X_e1)
X_e2 = scaler1.transform(X_e2)

# Step 3: Combine normalized data into a 3D array
X=np.zeros((X_e1.shape[0],X_e1.shape[1],2))
X[:,:,0]=X_e1
X[:,:,1]=X_e2

print("Shape of X_e1:", X_e1.shape)
print("Shape of X_e2:", X_e2.shape)
print("Shape of X:",X.shape)

In [ ]:
# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
x_train_new=x_train.copy()
x_test_new = x_test.copy()
# Convert labels to categorical
Y_train_new = tf.keras.utils.to_categorical(y_train)
Y_test_new = tf.keras.utils.to_categorical(y_test)
print("Shape of y_train:",y_train.shape)
print("Shape of Y_train_new:", Y_train_new.shape)

In [ ]:
# Reshape the training and testing data for CNN input
x_train_new=x_train_new.reshape(x_train_new.shape[0],  x_train_new.shape[1], 2)
x_test_new = x_test_new.reshape(x_test_new.shape[0], x_test_new.shape[1],2 )
print (x_train_new.shape)

In [19]:
# Create a Sequential model
model = Sequential()

# Add LSTM layers
model.add(LSTM(128, return_sequences=True, input_shape=(5, 2), activation='relu'))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(32, return_sequences=True, activation='relu'))
model.add(LSTM(32, return_sequences=True, activation='relu'))
model.add(LSTM(16))

# Add Dense and Activation layers
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001,
                                        beta_1=0.9,
                                        beta_2=0.999,
                                        epsilon=1e-07,
                                        amsgrad=False,
                                        name='Adam'),
              metrics=['accuracy'])

In [ ]:
# Define the file path for saving the best weights
checkpoint_filepath = '/content/drive/MyDrive/Code_IA_Test/best_weights.h5'

# Create a ModelCheckpoint callback
checkpointer = ModelCheckpoint(filepath=checkpoint_filepath,
                               verbose=1,
                               monitor='val_loss',
                               mode='auto',
                               save_best_only=True)

# Train the model with callbacks
history = model.fit(x_train_new, Y_train_new,
                    epochs=100,
                    batch_size=32,
                    verbose=1,
                    validation_split=0.20,
                    callbacks=[checkpointer])

# save the entire model
# model.save('weights_LSTM.h5')

In [ ]:
# Evaluate the model on the test set
score = model.evaluate(x_test_new, Y_test_new, verbose=1)

# Display the test loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend( loc='upper left')
plt.show()

In [ ]:
predictions = model.predict(x_test_new, batch_size=128, verbose=2)
print ("Predicted data shape:", predictions.shape)
print("True labels shape:", Y_test_new.shape)

In [ ]:
# Set the custom threshold for positive predictions
threshold_confusion = 0.5
print ("\nConfusion matrix:  Custom threshold (for positive) of " +str(threshold_confusion))

# Initialize arrays to store predicted and true labels
y_pred = np.empty((predictions.shape[0]))
y_test = np.empty((predictions.shape[0]))

# Populate the arrays with predicted and true labels
for i in range(predictions.shape[0]):
    y_pred[i]=np.argmax(predictions[i])
    y_test[i]=np.argmax(Y_test_new[i])

# Calculate and display the confusion matrix
confusion = confusion_matrix(y_test,  y_pred)
print (confusion)

In [ ]:
# Calculate and display accuracy
accuracy = accuracy_score(y_test, y_pred)
print("The accuracy score on this random test-set is:", accuracy)

# Calculate and display recall
recall = recall_score(y_test, y_pred, average=None)
average_recall = sum(recall) / len(recall)
print("Recall:", recall, "The average recall is", average_recall)

# Calculate and display precision
precision = precision_score(y_test, y_pred, average=None)
average_precision = sum(precision) / len(precision)
print("Precision:", precision, "The average precision is", average_precision)

# Calculate and display F1 score
f1_score = (2 * average_precision * average_recall) / (average_precision + average_recall)
print("F1 Score:", f1_score)